In [1]:
import tensorflow as tf
from keras.applications import VGG16
from keras import models, layers
from keras.optimizers import RMSprop
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

In [10]:
# Завантаження датасету Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


x_train = x_train / 255.0
x_test = x_test / 255.0


# Expand dimensions to add the channel dimension (should be (num_samples, height, width, 1))
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

# Repeat the single channel to get 3 channels (RGB)
x_train = np.repeat(x_train, 3, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)

# Convert to tensors
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

# Resize the images to 48x48 for VGG16
x_train = tf.image.resize(x_train, (48, 48))
x_test = tf.image.resize(x_test, (48, 48))

In [13]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

datagen.fit(x_train)

In [15]:
# Завантаження базової моделі VGG16 з предтренованими вагами ImageNet
base_model = VGG16(input_shape=(48, 48, 3),
                                include_top=False,
                                weights="imagenet")

base_model.trainable = False

# Додавання власних повнозв'язних шарів
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(10, activation="softmax")
])

58889256/58889256 [==============================] - 0s 0us/step


In [17]:
model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [18]:
history = model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 135s 72ms/step - loss: 0.8358 - accuracy: 0.7265 - val_loss: 0.5459 - val_accuracy: 0.8043
Epoch 2/10
1875/1875 [==============================] - 136s 73ms/step - loss: 0.5259 - accuracy: 0.8126 - val_loss: 0.4781 - val_accuracy: 0.8276
Epoch 3/10
1875/1875 [==============================] - 136s 72ms/step - loss: 0.4724 - accuracy: 0.8314 - val_loss: 0.4471 - val_accuracy: 0.8377
Epoch 4/10
1875/1875 [==============================] - 137s 73ms/step - loss: 0.4420 - accuracy: 0.8412 - val_loss: 0.4292 - val_accuracy: 0.8459
Epoch 5/10
1875/1875 [==============================] - 137s 73ms/step - loss: 0.4211 - accuracy: 0.8475 - val_loss: 0.4131 - val_accuracy: 0.8521
Epoch 6/10
1875/1875 [==============================] - 137s 73ms/step - loss: 0.4057 - accuracy: 0.8539 - val_loss: 0.4024 - val_accuracy: 0.8570
Epoch 7/10
1875/1875 [==============================] - 135s 72ms/step - loss: 0.3921 - accuracy: 0.8594 - val_loss: 0

In [21]:
base_model.trainable = True
set_trainable = False
for layer in base_model.layers:
  if layer.name == "block5_conv1":
    set_trainable = True
  layer.trainable = set_trainable

In [22]:
fine_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(10, activation="softmax")
])

In [23]:
fine_model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [25]:
fine_history = fine_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 223s 118ms/step - loss: 0.3901 - accuracy: 0.8630 - val_loss: 0.3596 - val_accuracy: 0.8825
Epoch 2/10
1875/1875 [==============================] - 216s 115ms/step - loss: 0.2818 - accuracy: 0.9006 - val_loss: 0.2893 - val_accuracy: 0.9057
Epoch 3/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.2591 - accuracy: 0.9103 - val_loss: 0.3115 - val_accuracy: 0.8991
Epoch 4/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.2425 - accuracy: 0.9157 - val_loss: 0.2663 - val_accuracy: 0.9100
Epoch 5/10
1875/1875 [==============================] - 215s 114ms/step - loss: 0.2339 - accuracy: 0.9226 - val_loss: 0.3208 - val_accuracy: 0.9069
Epoch 6/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.2291 - accuracy: 0.9231 - val_loss: 0.2956 - val_accuracy: 0.9004
Epoch 7/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.2220 - accuracy: 0.9258 - val_

In [30]:
test_loss, test_accuracy = fine_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}")

313/313 [==============================] - 17s 55ms/step - loss: 0.3127 - accuracy: 0.9131
Test accuracy: 91.31


In [31]:
extend_model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(10, activation="softmax")
])



In [33]:
extend_model.compile(optimizer=RMSprop(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [34]:
ext_history = extend_model.fit(datagen.flow(x_train, y_train, batch_size=32),
                         epochs=10,
                         validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 217s 115ms/step - loss: 0.5898 - accuracy: 0.8027 - val_loss: 0.2930 - val_accuracy: 0.9058
Epoch 2/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.4364 - accuracy: 0.8496 - val_loss: 0.3278 - val_accuracy: 0.8998
Epoch 3/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.4164 - accuracy: 0.8556 - val_loss: 0.3071 - val_accuracy: 0.8969
Epoch 4/10
1875/1875 [==============================] - 214s 114ms/step - loss: 0.4092 - accuracy: 0.8581 - val_loss: 0.2981 - val_accuracy: 0.9010
Epoch 5/10
1875/1875 [==============================] - 216s 115ms/step - loss: 0.4132 - accuracy: 0.8588 - val_loss: 0.2840 - val_accuracy: 0.9072
Epoch 6/10
1875/1875 [==============================] - 215s 115ms/step - loss: 0.4134 - accuracy: 0.8593 - val_loss: 0.3457 - val_accuracy: 0.8992
Epoch 7/10
1875/1875 [==============================] - 215s 114ms/step - loss: 0.4157 - accuracy: 0.8602 - val_

In [35]:
test_loss, test_accuracy = extend_model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy * 100:.2f}")

313/313 [==============================] - 17s 55ms/step - loss: 0.3881 - accuracy: 0.8898
Test accuracy: 88.98


# Висновок:

Точність моделі що наслідувала від VGG16 становить 91.31%
Точність моделі що навчалась додатково отримала результат у 88.98%